In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
import pandas as pd

# Define file paths
file_paths = {
    'shoppers': 'shoppers.csv',
    'offers': 'offers.csv',
    'engagements': 'engagements.csv',
    'shopper_offers': 'shopper_offers.csv'
}

# Load datasets with low_memory mode
shoppers = pd.read_csv(file_paths['shoppers'], low_memory=True)
offers = pd.read_csv(file_paths['offers'], low_memory=True)
shopper_offers = pd.read_csv(file_paths['shopper_offers'], low_memory=True)

# Load large file (engagements) in chunks
engagement_chunks = pd.read_csv(file_paths['engagements'], chunksize=5000000, low_memory=True)  # 5M rows per chunk

# Normalize column names (remove spaces, lowercase)
for df_name, df in zip(file_paths.keys(), [shoppers, offers, shopper_offers]):
    df.columns = df.columns.str.strip().str.lower()

# --- Function: Reduce Memory Usage ---
def optimize_memory(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# Apply memory optimization
shoppers = optimize_memory(shoppers)
offers = optimize_memory(offers)
shopper_offers = optimize_memory(shopper_offers)

# --- Processing Engagements in Chunks ---
processed_engagements = []
for chunk in engagement_chunks:
    chunk.columns = chunk.columns.str.strip().str.lower()  # Normalize column names
    chunk = optimize_memory(chunk)  # Reduce memory usage
    processed_engagements.append(chunk)  # Store optimized chunk

# Concatenate all processed chunks into a single dataframe
engagements = pd.concat(processed_engagements, ignore_index=True)

# --- Handle Non-Numeric Values in Large Dataset ---
def clean_numeric_column(df, column):
    if column in df.columns:
        df[column] = df[column].astype(str).str.split('|').str[0]  # Extract first number if pipe-separated
        df[column] = pd.to_numeric(df[column], errors='coerce')  # Convert safely
    return df

# Apply numeric cleaning only where necessary
shoppers = clean_numeric_column(shoppers, 'receipt_id')
shoppers = clean_numeric_column(shoppers, 'redemption_count')
engagements = clean_numeric_column(engagements, 'offer_id')

# --- Convert Date Columns Safely ---
date_columns = {
    'shoppers': 'date_verified',
    'engagements': 'event_at',
    'shopper_offers': 'registration_date'
}

for df_name, col in date_columns.items():
    if col in locals()[df_name].columns:
        locals()[df_name][col] = pd.to_datetime(locals()[df_name][col], errors='coerce')

# --- Handle Missing Values ---
shoppers.fillna({'receipt_id': -1, 'redemption_count': 0}, inplace=True)
shopper_offers.fillna({'registration_device': 'Unknown'}, inplace=True)

# --- Final Memory Optimization ---
shoppers = optimize_memory(shoppers)
offers = optimize_memory(offers)
engagements = optimize_memory(engagements)
shopper_offers = optimize_memory(shopper_offers)

# --- Final Check ---
print("✅ Data loading complete. Summary of memory usage:")
print("Shoppers:\n", shoppers.memory_usage(deep=True))
print("Offers:\n", offers.memory_usage(deep=True))
print("Engagements:\n", engagements.memory_usage(deep=True))
print("Shopper Offers:\n", shopper_offers.memory_usage(deep=True))

In [4]:
# Inspect DataFrames
# Inspect 'shoppers' DataFrame
print(shoppers.head())
print(shoppers.info())

# Repeat for other DataFrames

           customer_id             date_verified  retailer_id  offer_id  \
0  2393496587589073165  2018-04-28T21:55:39.000Z         32.0   1278627   
1  7213892974194484858  2018-03-08T21:26:52.000Z          4.0   1224026   
2  1531082007435697684  2018-04-28T20:23:46.000Z          9.0   1263179   
3  1531082007435697684  2018-06-25T02:42:44.000Z       1408.0   1321491   
4  2464622852289673103  2018-06-29T05:47:31.000Z         12.0   1298744   

                      receipt_id  redemption_count  
0                     1446779487                 1  
1                     1411515095                 1  
2                     1446708239                 1  
3  200010916620679|1687660627894                 1  
4                     1490601205                 1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413636 entries, 0 to 413635
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   customer_id       41

In [15]:
# Check missing values
print("Missing values in shoppers dataset:\n", shoppers.isnull().sum(), "\n")
print("Missing values in offers dataset:\n", offers.isnull().sum(), "\n")
print("Missing values in engagements dataset:\n", engagements.isnull().sum(), "\n")
print("Missing values in shopper_offers dataset:\n", shopper_offers.isnull().sum(), "\n")

Missing values in shoppers dataset:
 customer_id         0
date_verified       0
retailer_id         2
offer_id            0
receipt_id          2
redemption_count    0
dtype: int64 

Missing values in offers dataset:
 offer_id         0
reward_amount    0
offer_type       0
dtype: int64 

Missing values in engagements dataset:
 customer_id           0
event_at              0
session_id            0
event_type            0
retailer_id    10466766
offer_id       15114463
dtype: int64 

Missing values in shopper_offers dataset:
 customer_id               0
registration_date         0
registration_device    1424
dtype: int64 



In [17]:
# Handle missing values in shoppers dataset
shoppers = shoppers.dropna(subset=['registration_date'])  # Critical data, must not be missing

# Handle missing values in offers dataset
offers['offer_type'].fillna("Unknown", inplace=True)  # Categorical fill

# Handle missing values in engagements dataset
engagements = engagements.dropna(subset=['event_at'])  # Timestamps should not be missing

# Handle missing values in shopper_offers dataset
shopper_offers['redemption_count'].fillna(0, inplace=True)  # Replace missing redemption counts with 0

KeyError: ['registration_date']